# PydanticAI

In [1]:
## PydanticAI is asynchon
import asyncio

async def func():
    print("started.")
    await asyncio.sleep(2)
    print("done.")

await func()

started.
done.


In [120]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel

from openai import AsyncOpenAI

client = AsyncOpenAI(api_key="ollama", base_url="http://localhost:11434/v1")

model = OpenAIModel("llama3.2:latest", openai_client=client)

class CityLocation(BaseModel):
    city: str = Field(description="name of the city")
    zip_code: str = Field(description="zip code of the city in ISO 3166")
    country: str = Field(description="name of the country")
    geo_coordinates: str = Field(description="geo coordinates")

agent = Agent(model=model, result_type=CityLocation, system_prompt="You are a helpful asistent. Responding very precise.")

#async def run(prompt):
#    return await agent.run(prompt)

prompt = "In which city and country did the olympics take place in 2016?"

result = await agent.run(prompt)

In [121]:
result.data

CityLocation(city='Rio de Janeiro', zip_code='', country='Brazil', geo_coordinates='')

## Result
- all_messages
- all_messages_json
- cost
- data
- new_messages
- new_messages_json

In [122]:
print(result.cost().details)
print(result.cost().request_tokens)
print(result.cost().response_tokens)
print(result.cost().total_tokens)

None
238
36
274


In [123]:
import pydantic_ai
for m in result.all_messages():
    print(type(m),":",end=" ")
    if type(m) ==  pydantic_ai.messages.SystemPrompt:
        print(m.content)
    if type(m) ==  pydantic_ai.messages.UserPrompt:
        print(m.content)
    if type(m) ==  pydantic_ai.messages.ModelStructuredResponse:
        print(m.calls)
    if type(m) ==  pydantic_ai.messages.ToolReturn:
        print(m.content)

<class 'pydantic_ai.messages.SystemPrompt'> : You are a helpful asistent. Responding very precise.
<class 'pydantic_ai.messages.UserPrompt'> : In which city and country did the olympics take place in 2016?
<class 'pydantic_ai.messages.ModelStructuredResponse'> : [ToolCall(tool_name='final_result', args=ArgsJson(args_json='{"city":"Rio de Janeiro","country":"Brazil","geo_coordinates":"","zip_code":""}'), tool_id='call_0w6rzzn5')]
<class 'pydantic_ai.messages.ToolReturn'> : Final result processed.


In [124]:
import pydantic_ai
for m in result.new_messages():
    print(type(m),":",end=" ")
    if type(m) ==  pydantic_ai.messages.SystemPrompt:
        print(m.content)
    if type(m) ==  pydantic_ai.messages.UserPrompt:
        print(m.content)
    if type(m) ==  pydantic_ai.messages.ModelStructuredResponse:
        print(m.calls)
    if type(m) ==  pydantic_ai.messages.ToolReturn:
        print(m.content)

<class 'pydantic_ai.messages.UserPrompt'> : In which city and country did the olympics take place in 2016?
<class 'pydantic_ai.messages.ModelStructuredResponse'> : [ToolCall(tool_name='final_result', args=ArgsJson(args_json='{"city":"Rio de Janeiro","country":"Brazil","geo_coordinates":"","zip_code":""}'), tool_id='call_0w6rzzn5')]
<class 'pydantic_ai.messages.ToolReturn'> : Final result processed.


In [125]:
print(type(result.data))
print(result.data)

<class '__main__.CityLocation'>
city='Rio de Janeiro' zip_code='' country='Brazil' geo_coordinates=''


In [126]:
## Use Tool Calling
import requests
from bs4 import BeautifulSoup

async def get_zip_code(city: str, country: str) -> str:
    zip_code = None
    geo_cordinates = None
    search_string = "+".join([item.lower() for item in city.split(" ")])
    #print(search_string)
    url = f'http://www.geonames.org/postalcode-search.html?q="{search_string}"'

    # A GET request to the API
    response = requests.get(url)

    if response.ok:
        soup = BeautifulSoup(response.text, "html.parser")
        response_tables = soup.find_all(class_='restable')
        response_tables = response_tables[0]    
        rows =  response_tables.find_all(name='tr')
        # hack (because html is not well formatted)
        if len(rows) > 0:
            rows = str(rows[1]).split("</a></td></tr>")
            for row in rows:
                row = row.replace("<tr><td>","") # bug
                soup = BeautifulSoup(row, "html.parser")
                columns = soup.getText(separator=';', strip=True)
                columns = columns.split(";")
                if city == columns[1] and country == columns[3]:
                    zip_code = columns[2]
                    geo_cordinates = columns[6]
                break
    return zip_code, geo_cordinates

In [128]:
#city = 'Rio de Janeiro'
#country = 'Brazil'
#zip_code, geo_cordinates = await get_zip_code(city, country)
#print(zip_code, geo_cordinates)

20000-000 -22.92/-43.331


In [153]:
from dataclasses import dataclass

@dataclass
class Deps:
    weather_api_key: str | None
    geo_api_key: str | None

city_agent = Agent(model=model, result_type=CityLocation, 
                   system_prompt="You are a helpful asistent. Responding very precise.",
                   retries=2, 
                   deps_type=Deps)



In [154]:
## Use Tool Calling
import requests
from bs4 import BeautifulSoup
from pydantic_ai import RunContext

@city_agent.tool
async def get_zip_code(ctx: RunContext[Deps],
                       city: str, country: str) -> dict[str,str]:
    """Gets the zip_code of a city and its geo coordinates.

    Args:
        ctx: The context
        city: The name of the city
        country: The name of the country
    """
    print(city, country)
    zip_code = None
    geo_cordinates = None
    search_string = "+".join([item.lower() for item in city.split(" ")])
    #print(search_string)
    url = f'http://www.geonames.org/postalcode-search.html?q="{search_string}"'

    # A GET request to the API
    response = requests.get(url)

    if response.ok:
        soup = BeautifulSoup(response.text, "html.parser")
        response_tables = soup.find_all(class_='restable')
        response_tables = response_tables[0]    
        rows =  response_tables.find_all(name='tr')
        # hack (because html is not well formatted)
        if len(rows) > 0:
            rows = str(rows[1]).split("</a></td></tr>")
            for row in rows:
                row = row.replace("<tr><td>","") # bug
                soup = BeautifulSoup(row, "html.parser")
                columns = soup.getText(separator=';', strip=True)
                columns = columns.split(";")
                if city == columns[1] and country == columns[3]:
                    zip_code = columns[2]
                    geo_cordinates = columns[6]
                    print(zip_code, geo_cordinates)
                break
    return {"zip_code": zip_code, "geo_cordinates": geo_cordinates}

In [157]:
#async def run(prompt):
#    return await agent.run(prompt)

#prompt = "In which city and country did the olympics take place in 2016?"
prompt = "What are the zip code and the geo coordintaes of Rio de Janeiro in Brazil?"

result = await city_agent.run(prompt)

Rio de Janeiro Brazil
20000-000 -22.92/-43.331


In [159]:
result.data

CityLocation(city='Rio de Janeiro', zip_code='20000-000', country='Brazil', geo_coordinates='-22.92/-43.33')

In [ ]:
# use https://www.python-httpx.org
# use Weather API: https://openweathermap.org/api/geocoding-api#direct_zip_how